In [1]:
from IPython.display import Audio
import shutil
import os
import time
import librosa
from pydub import AudioSegment
import numpy as np

In [2]:
start_time = time.time()

# Tách các tệp âm thanh bằng mô hình được đào tạo trước
!spleeter separate -o output ./input/noise_reduction_input.wav

end_time = time.time()
processing_time = end_time - start_time

INFO:spleeter:File output\noise_reduction_input/accompaniment.wav written succesfully
INFO:spleeter:File output\noise_reduction_input/vocals.wav written succesfully


In [3]:
# Đổi tền, điều chỉnh đường dẫn file output
if os.path.exists("output/noise_reduction_input/accompaniment.wav"):
    os.remove("output/noise_reduction_input/accompaniment.wav")

if os.path.exists("output/noise_reduction_input/vocals.wav"):
    old_path = "output/noise_reduction_input/vocals.wav"
    new_path = "output/spleeter_output.wav"
    shutil.move(old_path, new_path)

if os.path.exists("output/noise_reduction_input"):
    shutil.rmtree("output/noise_reduction_input")

In [4]:
def calculate_cleanliness_ratio(original_audio, denoised_audio):
    # Chuyển đổi AudioSegment thành mảng numpy
    original_array = np.array(original_audio.get_array_of_samples())
    denoised_array = np.array(denoised_audio.get_array_of_samples())

    # Đảm bảo rằng kích thước của hai mảng là giống nhau hoặc có thể broadcast cho nhau
    min_len = min(len(original_array), len(denoised_array))
    original_array = original_array[:min_len]
    denoised_array = denoised_array[:min_len]

    # Tính toán công suất tín hiệu và công suất nhiễu
    signal_power = np.sum(original_array ** 2)
    noise_power = np.sum((original_array - denoised_array) ** 2)

    # Tính toán SNR
    snr = 10 * np.log10(signal_power / noise_power)

    # Tính toán tỷ lệ sạch dưới dạng phần trăm
    cleanliness_ratio = (1 - 10**(-snr / 10)) * 100

    return cleanliness_ratio

# Đọc file âm thanh gốc và sau khi giảm ồn
original_audio = AudioSegment.from_file("./input/noise_reduction_input.wav")
denoised_audio = AudioSegment.from_file("output/spleeter_output.wav")

# Tính toán tỷ lệ sạch
cleanliness_ratio = calculate_cleanliness_ratio(original_audio, denoised_audio)


In [5]:
print(f"Thời gian xử lý: {processing_time:.2f} giây")
print(f"Tỷ lệ sạch: {cleanliness_ratio:.2f}%")

Thời gian xử lý: 15.28 giây
Tỷ lệ sạch: 18.51%
